# Sequence Pediction Metrics

Sequence prediction metrics all seek to summarize and quantify the extent to which a model has managed to reproduce, or accurately match, some gold standard sequences. Such problem arise throughout NLP.

Examples:

1. Mapping speech signals to their desired transcriptions.
1. Mapping texts in a language $L_{1}$ to their translations in a distinct language or dialect $L_{2}$.
1. Mapping input dialogue acts to their desired responses.
1. Mapping a sentence to one of its paraphrases.
1. Mapping real-world scenes or contexts (non-linguistic) to descriptions of them (linguistic).

Evaluations is very challenging because the relationships tend to be __many-to-one__: a given sentence might have multiple suitable translations; a given dialogue act will always have numerous felicitous responses; any scene can be described in multiple ways; and so forth. The most constrained of these problems is the speech-to-text case in 1, but even that one has indeterminacy in real-world contexts (humans often disagree about how to transcribe spoken language).

## Contents

- **Word error rate**
- **BLUE score**
- **Perplexity**


## Imports

In [1]:
%matplotlib inline
from nltk.metrics.distance import edit_distance
from nltk.translate import bleu_score
import numpy as np
import pandas as pd
import scipy.stats
from sklearn import metrics

## Word Error Rate

The [word error rate](https://en.wikipedia.org/wiki/Word_error_rate) (WER) metric is a word-level, length-normalized measure of [Levenshtein string-edit distance](https://en.wikipedia.org/wiki/Levenshtein_distance):


In [2]:
def wer(seq_true, seq_pred):
    d = edit_distance(seq_true, seq_pred)
    return d / len(seq_true)

In [3]:
wer(['A', 'B', 'C'], ['A', 'A', 'C'])

0.3333333333333333

In [4]:
wer(['A', 'B', 'C', 'D'], ['A', 'A', 'C', 'D'])

0.25

To calculate this over the entire test-set, one gets the edit-distances for each gold–predicted pair and normalizes these by the length of all the gold examples, rather than normalizing each case:

In [5]:
def corpus_wer(y_true, y_pred):
    dists = [edit_distance(seq_true, seq_pred)
             for seq_true, seq_pred in zip(y_true, y_pred)]
    lengths = [len(seq) for seq in y_true]
    return sum(dists) / sum(lengths)

This gives a single summary value for the entire set of errors.

### Bounds of word error rate

$[0, \infty)$, where 0 is best. (The lack of a finite upper bound derives from the fact that the normalizing constant is given by the true sequences, and the predicted sequences can differ from them in any conceivable way in principle.)

### Value encoded by word error rate

This method says that our desired notion of closeness or accuracy can be operationalized in terms of the low-level operations of insertion, deletion, and substitution. The guiding intuition is very much like that of F scores.

### Weaknesses of word error rate

The value encoded reveals a potential weakness in certain domains. Roughly, the more __semantic__ the task, the less appropriate WER is likely to be. 

For example, adding a negation to a sentence will radically change its meaning but incur only a small WER penalty, whereas passivizing a sentence (_Kim won the race_ &rarr; _The race was won by Kim_) will hardly change its meaning at all but incur a large WER penalty. 

See also [Liu et al. 2016](https://www.aclweb.org/anthology/D16-1230) for similar arguments in the context of dialogue generation.

### Related to word error rate

* WER can be thought of as a family of different metrics varying in the notion of edit distance that they employ.

* The Word Accuracy Rate is 1.0 minus the WER, which, despits its name, is intuitively more like [recall](#Recall) than [accuracy](#Accuracy).

## BLEU Scores

BLEU(Bilingual Evaluation Understudy) scores were originally developed in the context of machine translation, but they are applied in other generation tasks as well.

For BLEU scoring, we require a dataset $Y$ consisting of instances $(a, B)$ where $a$ is a candidate (a model prediction) and $B$ is a set of gold texts. The metric has two main components:

* __Modified n-gram precision__: A direct application of precision would divide the number of correct n-grams in the candidate (n-grams that appear in any translation) by the total number of n-grams in the candidate This has a degenerate solution in which the predicted output contains only one n-gram. BLEU's modified version substitutes the actual count for each n-gram $s$ in the candidate by the maximum number of times $s$ appears in any gold text.

* __Brevity penalty (BP)__: to avoid favoring outputs that are too short, a penalty is applied. Let $r$ be the sum of all minimal absolute length differences between candidates and referents in the dataset $Y$, and let $c$ be the sum of the lengths of all the candidates. Then:

$$\textbf{BP}(Y) =
\begin{cases}
1 & \textrm{ if } c > r \\
\exp(1 - \frac{r}{c}) & \textrm{otherwise}
\end{cases}$$



The BLEU score itself is typically a combination of modified n-gram precision for various $n$ (usually up to 4):

$$\textbf{BLEU}(Y) = \textbf{BP}(Y) \cdot 
    \exp\left(\sum_{n=1}^{N} w_{n} \cdot \log\left(\textbf{modified-precision}(Y, n\right)\right)$$

where $Y$ is the dataset, and $w_{n}$ is a weight for each $n$-gram level (usually set to $1/n$).

NLTK has [a flexible implementation of Bleu scoring](http://www.nltk.org/_modules/nltk/translate/bleu_score.html).

### Bounds of BLEU scores

[0, 1], with 1 being the best, though with no expectation that any system will achieve 1, since even sets of human-created translations do not reach this level.

### Value encoded by BLEU scores

BLEU scores attempt to achieve the same balance between precision and recall that runs through the majority of the metrics discussed here. It has many affinities with [word error rate](#Word-error-rate), but seeks to accommodate the fact that there are typically multiple suitable outputs for a given input.

### Weaknesses of BLEU scores

* [Callison-Burch et al. (2006)](http://www.aclweb.org/anthology/E06-1032) criticize BLEU as a machine translation metric on the grounds that it fails to correlate with human scoring of translations. They highlight its insensitivity  to n-gram order and its insensitivity to n-gram types (e.g., function vs. content words) as causes of this lack of correlation.

* [Liu et al. (2016)](https://www.aclweb.org/anthology/D16-1230) specifically argue against BLEU as a metric for assessing dialogue systems, based on a lack of correlation with human judgments about dialogue coherence.

### Related to BLEU scores

There are many competitors/alternatives to BLEU, most proposed in the context of machine translation. Examples: [ROUGE](https://en.wikipedia.org/wiki/ROUGE_(metric), [METEOR](https://en.wikipedia.org/wiki/METEOR), [HyTER](http://www.aclweb.org/anthology/N12-1017), [Orange (smoothed Bleu)](http://www.aclweb.org/anthology/C04-1072).

## Perplexity

[Perplexity](https://en.wikipedia.org/wiki/Perplexity) is a common metric for directly assessing generation models by calculating the probability that they assign to sequences in the test data:

$$\textbf{PP}(p, \textbf{x}) = \prod_{i=1}^{n}\left(\frac{1}{p(x_{i})}\right)^{\frac{1}{n}}$$

where $p$ is a model assigning probabilities to elements and $\textbf{x}$ is a sequence of length $n$.

When averaging perplexity values obtained from all the sequences in a text corpus, one should again use the geometric mean:

$$\textbf{mean-PP}(p, X) = 
\exp\left(\frac{1}{m}\sum_{x\in X}\log \textbf{PP}(p, \textbf{x})\right)$$

for a set of $m$ examples $X$.

### Bounds of perplexity

[1, $\infty$], where 1 is best.

### Values encoded by perplexity

The guiding idea behind perplexity is that a good model will assign high probability to the sequences in the test data. This is an intuitive, expedient intrinsic evaluation, and it matches well with the objective for models trained with a cross-entropy or logistic objective.

### Weaknesses of perplexity

* Perplexity is heavily dependent on the nature of the underlying vocabulary in the following sense: one can artificially lower one's perplexity by having a lot of `UNK` tokens in the training and test sets. Consider the extreme case in which _everything_ is mapped to `UNK` and perplexity is thus perfect on any test set. The more worrisome thing is that any amount of `UNK` usage side-steps the pervasive challenge of dealing with infrequent words.

* [As Hal Daumé discusses in this post](https://nlpers.blogspot.com/2014/05/perplexity-versus-error-rate-for.html), the perplexity metric imposes an artificial constraint that one's model outputs are probabilistic.

### Related to perplexity

Perplexity is the inverse of probability and, [with some assumptions](http://www.cs.cmu.edu/~roni/11761/PreviousYearsHandouts/gauntlet.pdf), can be seen as an approximation of the cross-entropy between the model's predictions and the true underlying sequence probabilities.